In [29]:
import numpy as np
import pandas as pd

In [30]:
def load_data():
    raw_df = []
    dict_df = pd.read_excel(r'C:\Users\SR-19\Desktop\Analog_Reports\Solar data\GridTieInverterData.xlsx',skiprows=5,parse_dates={'Time':[2]},index_col='Time',skipfooter=1,sheet_name=None)
    for key,dframe in dict_df.items():
        dframe.drop(dframe.tail(1).index,inplace=True)
        raw_df.append(dframe[['PDC(W)']])
    raw_df=pd.concat(raw_df)
    raw_df.index=pd.to_datetime(raw_df.index,format='%Y-%m-%d %H:%M:%S')
    raw_df.sort_index(inplace=True,ascending=False)
    return raw_df

In [4]:
def load_weather():
    weather = pd.read_csv(r'C:\Users\SR-19\Desktop\Analog_Reports\kanpur_weather_solar.csv',
                             header=24)
    LIST=[]
    for i in range(weather.shape[0]):
        if weather.HR.values[i] < 10:
            string = f"{weather.YEAR.values[i]}-{weather.MO.values[i]}-{weather.DY.values[i]} {0}{weather.HR.values[i]}"
        else:
            string = f"{weather.YEAR.values[i]}-{weather.MO.values[i]}-{weather.DY.values[i]} {weather.HR.values[i]}"

        LIST.append(pd.to_datetime(string,format='%Y-%m-%d %H'))

    weather['time']=LIST
    weather=weather.set_index(weather['time'],drop=False)
    weather.drop(['YEAR','MO','DY','HR','time'],axis=1,inplace=True)
    return weather

,Temperature(C),PDC(W),PAC(W)
Time,,,
2021-01-01 00:39:32.377,4.6,0.0,0.0
2021-01-01 00:29:53.340,4.6,0.0,0.0
2021-01-01 00:20:14.380,4.6,0.0,0.0
2021-01-01 00:10:35.433,4.6,0.0,0.0
2021-01-01 00:00:56.390,4.6,0.0,0.0


In [31]:
raw_df= load_data()

In [32]:
def clean_data(df):
    df['datetime'] = [str(index)[0:13] for index in df.index.values]
    df = df.groupby('datetime').mean()
    df.index=pd.to_datetime(df.index,format='%Y-%m-%d %H')
    return df

In [ ]:
def fill_missing(df):
    start,end=raw_df.index[0],raw_df.index[-1]
    index=pd.date_range(start=start,end=end,freq='1H')
    t_df=pd.DataFrame({'val':np.NaN},index=index)
    result = raw_df.join(t_df, how="outer").drop(['val'],axis=1)
    zero_pow = [19,20,21,22,23,0,1,2,3,4]
    for index in result.index:
        if index.hour in zero_pow:
            result.loc[index]=0
        else: 
    return result